In [1]:
# Creating file with only Auth, Assoc, and Deauth keywords
# REPLACE FILE NAMES AS NEEDED
with open('wireless_01-01-2021.log', 'r') as cleaned_test_file, open('cleaned_test_file2.txt', 'a') as cleaned_test_file2:    
    unwanted_strings = " wifi_deauth_sta"+" Response to EAP Challenge Failed"+" APAE Disconnect"+" handle_sapcp"+" Sapcp Ageout (internal ageout) "+" Capability requested by STA unsupported by AP"+" STA has roamed to another AP"
    
    for line in cleaned_test_file.readlines():
        # Getting only Auth and Assoc lines
        if "success" in line:
            cleaned_test_file2.write(line)
        
        # Getting Deauth lines
        if "Deauth" in line:
            newstring = line.replace("Ageout ", "")
            cleaned_test_file2.write(newstring.strip(unwanted_strings + "\n") + "\n")

In [12]:
# Matching Access Points in Auth, Assoc, and Deauth lines; Removing lines w/ repeating Access Points
with open('cleaned_test_file2.txt', 'r') as cleaned_test_file2, open('cleaned_test_file5.txt', 'a') as cleaned_test_file5:
    mac_ip_list = []
    temp_mac_ip_list = []
    line_array = []
    
    for line in cleaned_test_file2.readlines():
        # Getting Auth lines
        if "Auth s" in line:
            line_words = line.split()
            mac_and_IP_auth = line_words[-3] + ' AP ' + line_words[-1]
            
            # Conditions to get Auth lines (preventing repeats of same Auth lines)
            if mac_and_IP_auth not in temp_mac_ip_list:
                temp_mac_ip_list.append(mac_and_IP_auth)
                line_array.append(line)
                
        # Getting Assoc lines
        if "Assoc" in line:
            line_words = line.split()
            mac_and_IP_assoc = line_words[-3] + ' AP ' + line_words[-1]
                    
            # Preventing repeats of same Assoc lines, and matching to corresponding Auth lines
            if mac_and_IP_assoc in temp_mac_ip_list and mac_and_IP_assoc not in mac_ip_list:
                line_index = temp_mac_ip_list.index(mac_and_IP_assoc)
                auth_line = line_array[line_index]
                
                cleaned_test_file5.write(auth_line)
                cleaned_test_file5.write(line)
                mac_ip_list.append(mac_and_IP_assoc)
                
        # Getting Deauth lines
        if "Deauth" in line:
            line_words = line.split()
            mac_and_IP = line_words[-3] + ' AP ' + line_words[-1]
            
            # Getting Deauth lines only if there are Auth and Deauth lines
            if mac_and_IP in mac_ip_list:
                line_index = temp_mac_ip_list.index(mac_and_IP)
                
                # After Deauth, stored info no longer needed for later usage
                temp_mac_ip_list.remove(mac_and_IP)
                line_array.remove(line_array[line_index])       
                mac_ip_list.remove(mac_and_IP)
                
                cleaned_test_file5.write(line)